                                                😁  Bem vindo! 😁

#### Meu nome é Luis Trindade e sócio fundador da empresa Iteratec Desenvolvimento de Software!

#### Este notebook é material complementar para o curso disponível no youtube: https://www.youtube.com/playlist?list=PLhJbeYspP-A-Dj8Te8dUtf5Q9dprQuZk8

# Criando nossa base de dados 📖

In [3]:
import os
from PIL import Image, ImageDraw, ImageFile
from random import random
import shutil

def build_dir(list_of_folders):
    for item in path:
        try:
            os.listdir(f'../images/{item}')
        except:
            os.mkdir(f'../images/{item}')

def create_dataset():
    for file in os.listdir('../images'):
        print(file)
        if(file.endswith(".png")):
            im=Image.open('../images/' + file)
            rgb_im = im.convert('RGB')
            imResize = rgb_im.resize((256,256), Image.ANTIALIAS)
            imResize.save('../images/mod_'+file)  
            rand=random()
            if(rand<=0.85):
                shutil.move('../images/mod_' + file,'../images/train/' + file)
            else:
                shutil.move('../images/mod_' + file,'../images/test/' + file)


In [4]:
path=['train','test']
build_dir(path)

In [5]:
# create_dataset()

# Preparando nosso dataset 📊

### Funções Auxiliares 🧪

In [ ]:
#Importando nossos pacotes e funções:

import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import math
import json
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


#Preprocessor - One hot encoder --> Transforma uma variável discreta em múltiplas variáveis binárias.
def preprocessor_disease(df):
    
    categories=df.Disease.unique()
    ohe = OneHotEncoder(drop='if_binary', sparse = False)
    ohe.fit(df[['Disease']]) # Fit encoder
    diseases_encoaded = ohe.transform(df[['Disease']])
    categories.sort()
    for index,item in enumerate(categories):
        df[item]=diseases_encoaded[:,index]    
    return df

#Create csv file --> Cria um arquivo do tipo .csv contendo o nome do arquivo com e sem extensão e a doença presente
#no título do arquivo.
def create_data_csv(path, folder):
    list_images=[]
    for file in os.listdir(path):
        list_images.append({"filename":os.path.splitext(file)[0],"Disease":(os.path.splitext(file)[0]).split("-")[0]})
    df=pd.DataFrame(list_images)
    df=preprocessor_disease(df)
    try:
        df.drop(index=df.loc[df['filename']==folder+'_df'].index[0],inplace=True)
        df.drop(columns=folder+'_df',inplace=True)
    except:
        pass
    df.to_csv(path+ '/' + folder + '_df.csv', index=False)
    
#Plot distribution --> Cria um gráfico de barras contendo as categorias, ou, doenças, no eixo X
#e a contagem de ocorrências no eixo Y.    
def plot_dist(df,categories,position,reference):  
    x_axis=[]
    y_axis=[]
    for index,item in enumerate(categories):
        x_axis.append(item)
        y_axis.append(df.loc[df[item]==1].count()[0])
    sns.set_theme(style="whitegrid")
    sns.barplot(x=x_axis,y=y_axis,ax=axs[position]).set(title=reference)
    sns.lineplot(x=x_axis,y=np.average(y_axis),ax=axs[position])

### Processamento principal🔥

In [ ]:
#Definindo o caminho das pastas train e test.
path=['../images/train','../images/test']
folder=['train','test']

#Criando arquivos do tipo .csv utilizando a função definida nas funções auxiliares.
for index,item in enumerate(path):
    create_data_csv(path[index],folder[index])
    
#Transformando o conteúdo do arquivo .csv em um Pandas DataFrame.
train_df=pd.read_csv(path[0]+ '/' + folder[0] + '_df.csv')
test_df=pd.read_csv(path[1]+ '/' + folder[1] + '_df.csv')

#Definindo uma porcentagem aproximada do DataFrame que será utilizada no treinamento.
n=0.1 #Para trabalharmos apenas com 10% do dataset

#Realizando a abertura das imagens e transformando as mesmas em uma lista de matrizes.
X_train=[]
Y_train=[]
for file in os.listdir('../images/train'):
    rand=random()
    if(rand<=0.1):
        if(file.endswith(".png")):
            im=Image.open('../images/train/' + file)
            X_train.append(np.asarray(im))
            im_row=train_df[train_df['filename']==os.path.splitext(file)[0]].drop(columns="filename")
            Y_train.append(im_row.drop(columns='Disease').to_numpy()[0])

#Transformando a lista de matrizes em uma estrutura do tipo Numpy Array.
X_train=np.asarray(X_train)
#Normalizando as matrizes que compõem a imagem cujos valores encontram-se entre 0 e 255 para o intervalo de 0 e 1.
X_train=X_train/255
#Transformando nossa lista de listas em uma estrutura do tipo Numpy Array.
Y_train=np.asarray(Y_train)

#Embaralhando nosso ambiente de treinamento de forma a garantir que as doenças estejam distribuidas ao acaso.
X_train, Y_train = shuffle(X_train, Y_train)
#Separando nosso ambiente de treinamento do ambiente de validação em 70% dos dados para treinamento e 30% para validação.
div=math.floor(len(X_train)*0.7)
X_tr = X_train[:div]
y_tr = Y_train[:div]
X_val = X_train[div:]
y_val = Y_train[div:]

# Avaliando nosso dataset 🔍 

### Funções Auxiliares 🧪

In [ ]:
#Plot distribution --> Cria um gráfico de barras contendo as categorias, ou, doenças, no eixo X
#e a contagem de ocorrências no eixo Y.    
def plot_dist(df,categories,position,reference):  
    x_axis=[]
    y_axis=[]
    for index,item in enumerate(categories):
        x_axis.append(item)
        y_axis.append(df.loc[df[item]==1].count()[0])
    sns.set_theme(style="whitegrid")
    sns.barplot(x=x_axis,y=y_axis,ax=axs[position]).set(title=reference)
    sns.lineplot(x=x_axis,y=np.average(y_axis),ax=axs[position])

### Processamento principal🔥

In [ ]:
print(X_train.shape)
print(Y_train.shape)

#Realizando apresentações gráficas da nossa base de dados
f, axs = plt.subplots(1,2,figsize=(12,4))    
categories=['COVID','Lung_Opacity','Normal','Viral Pneumonia']
plot_dist(train_df,categories,0,'Amostra Original')
train_df=pd.DataFrame(Y_train,columns=['COVID','Lung_Opacity','Normal','Viral Pneumonia'])
plot_dist(train_df,categories,1,'Amostra contendo '+str(n*100)+'% da amostra original')

print(f"Nosso baseline para a nova amostragem é de aproximadamente {str(round(train_df.loc[train_df['Normal']==1].count()[0]/len(X_train)*100))}%.")
plt.show()

# Definindo nosso modelo 🤖

### Funções Auxiliares 🧪

In [ ]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

#Função de inicialização do modelo, indicando as camadas do modelo.
def initialize_model():
    
    model = Sequential()
    model.add(Conv2D(16, (2, 2), activation='relu', padding='same', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D((2, 2)))  
    model.add(Conv2D(16, (2, 2), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))  
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    return model

#Função de compilação do modelo
def compile_model(model):
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = adam,
                  metrics = metrics)
    return model

### Processamento principal🔥

In [ ]:
#Criando diretório onde iremos salvar os pesos de nosso modelo durante o treinamento
try:
    os.listdir('models')
    for file in os.listdir('models'):
        os.remove('models/'+file)
except:
    os.mkdir(f'models')

#Definindo a métrica de otimização do modelo.
metrics=['accuracy']

#Definindo a taxa de aprendizado e o nosso otimizador.
Learning_rate=0.01
adam = Adam(learning_rate = Learning_rate)

#Inicializando nosso modelo
model= initialize_model()
model.summary()

# Treinando nosso modelo 🦾

### Processamento principal🔥

In [ ]:
#Criando checkpoint para que os pesos do modelo sejam salvos a cada época do treinamento.
model_checkpoint=ModelCheckpoint('models/model{epoch:02d}.h5',save_weights_only=True)

#Realizando a compilação do modelo.
model = compile_model(model)

#Realizando o treinamento do nosso modelo.
history = model.fit(x=X_tr, y=y_tr, 
                    validation_data=(X_val,y_val),
                    use_multiprocessing=True,                    
                    batch_size=16,
                    epochs = 50, 
                    callbacks = [model_checkpoint])

# Avaliando a performance do nosso modelo 📈

### Funções Auxiliares 🧪

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#Função que irá realizar uma representação gráfica da curva de perda e acurácia do modelo.
def plot_loss_accuracy_curve(history):
    
    fig, ax =plt.subplots(1,2,figsize=(20,5))
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].legend(['Train', 'Val'], loc='upper right')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)

    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy', fontsize = 18)
    ax[1].set_xlabel('Epoch', fontsize = 14)
    ax[1].set_ylabel('Precision', fontsize = 14)
    ax[1].legend(['Train', 'Val'], loc='lower right')  
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)    
    
    plt.show()

#Função que irá realizar uma representação gráfica da matriz de confusão.
def plot_confusion_matrix(X,y,model):
    Y_pred=model.predict(X)
    Y_pred_cat=[]
    Y_cat=[]
    for item in Y_pred:
        Y_pred_cat.append(np.argmax(item))
    for item in y:
        Y_cat.append(np.argmax(item))
    
    cm = confusion_matrix(Y_cat, Y_pred_cat,normalize='true')
    cm_2 = confusion_matrix(Y_cat, Y_pred_cat)
    
    cm_df = pd.DataFrame(cm,
                     index = categories, 
                     columns = categories)

    cm2_df = pd.DataFrame(cm_2,
                     index = categories, 
                     columns = categories)
    
    plt.figure(figsize=(7,5))
    sns.heatmap(cm_df, annot=True)
    plt.figure(figsize=(7,5))
    sns.heatmap(cm2_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actal Values')
    plt.xlabel('Predicted Values')
    plt.show()
    
    print(classification_report(Y_cat,Y_pred_cat))

### Processamento principal🔥

In [ ]:
#Realizando a representação gráfica das curvas de perda e acurácia durante o treinamento.
plot_loss_accuracy_curve(history)

#Realizando a representação gráfica da matriz de confusão para o treinamento e validação.
plot_confusion_matrix(X_tr,y_tr,model)
plot_confusion_matrix(X_val,y_val,model)

# Salvando nosso modelo 💾

### Processamento principal🔥

In [ ]:
from tensorflow.keras.models import model_from_json
import glob

#Transformando o modelo em um arquivo .json para salvar.
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    print("Modelo salvo com sucesso")
    
#Carregando o modelo a partir de um arquivo do tipo .json.
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
#Carregando o peso da última época de treinamento do modelo.
list_of_files = glob.glob('models/*')
latest_file = max(list_of_files, key=os.path.getctime)
try:
    model.load_weights(latest_file)
    print("Modelo carregado com sucesso")
    print(f"Peso carregado está no diretório {latest_file}")
    compile_model(model)
except:
    print('Verifique se existe histórico de treinamento para o modelo.')

# Avaliando a performance do nosso modelo com imagens de teste 📈

### Funções Auxiliares 🧪

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score

#Função para criação de representação gráfica da curva de ROC.
def plot_roc_curve(fpr,tpr): 
    plt.plot(fpr,tpr) 
    x = np.arange(0, 1.1, 0.1).tolist()
    y = x
    plt.plot(x,y)
    plt.axis([0,1,0,1]) 
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    plt.show()    


### Processamento principal🔥

In [ ]:
X_test=[]
Y_test=[]

#Criando dataset de teste para avaliação do modelo em ambiente de teste.
for file in os.listdir('images/test'):
    if(file.endswith(".png")):
        im=Image.open('images/test/' + file)
        X_test.append(np.asarray(im))
        im_row=test_df[test_df['filename']==os.path.splitext(file)[0]].drop(columns="filename")
        Y_test.append(im_row.drop(columns='Disease').to_numpy()[0])
        
X_test=np.asarray(X_test)
X_test=X_test/255
Y_test=np.asarray(Y_test)

res = model.evaluate(X_test, Y_test, verbose = 1)

#Aferindo a acurácia do modelo em dados de teste.
print(f'Acurácia do modelo para os dados do diretório de teste:  {res[1]*100:.2f}%')

#Realizando representação gráfica da matriz de confusão das predições do modelo com dados de teste.
plot_confusion_matrix(X_test,Y_test,model)

#Realizando representação gráfica das curvas de ROC
f, axs = plt.subplots(2,4,figsize=(20,10))
bins = [i/20 for i in range(20)] + [1]
classes = categories
roc_auc_ovr = {}
Y_pred=model.predict(X_test)

for i in range(len(classes)):
    # Gets the class
    c = classes[i]
    
    df_aux=pd.DataFrame(Y_pred,columns=categories).join(test_df['Disease'])
    # Prepares an auxiliar dataframe to help with the plots
    df_aux["class"] = [1 if y == c else 0 for y in df_aux['Disease']]
    df_aux['prob'] = df_aux[c]

    # Plots the probability distribution for the class and the rest
    ax = plt.subplot(2, 4, i+1)
    sns.histplot(x = "prob", data = df_aux, hue = 'class', color = 'b', ax = ax, bins = bins)
    ax.set_title(c)
    ax.legend([f"Class: {c}", "Rest"])
    ax.set_xlabel(f"P(x = {c})")
    
    # Calculates the ROC Coordinates and plots the ROC Curves
    ax = plt.subplot(2, 4, i+5)
    fpr , tpr , thresholds = roc_curve (df_aux['class'], df_aux['prob'])
    plt.plot(fpr,tpr) 
    x = np.arange(0, 1.1, 0.1).tolist()
    y = x
    plt.plot(x,y)
    plt.axis([0,1,0,1]) 
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    ax.set_title("ROC Curve OvR")
    roc_auc_ovr[c] = roc_auc_score(df_aux['class'], df_aux['prob'])
    print(f"AUC for {c} is {roc_auc_ovr[c]}.")
